In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_source_file_path = 'csv_source_files/'

def weather_from_2020Nov_to_2021Feb (csvFile):
    weather = pd.read_csv(csv_source_file_path + csvFile)
    weather = weather[ (weather['Date/Time']>'2020-10-31') & (weather['Date/Time']<'2021-03-01') ]
    return weather

frames = [weather_from_2020Nov_to_2021Feb("en_climate_daily_ON_6106001_2020_P1D.csv"), # OTTAWA INTL A weather 2020 
          weather_from_2020Nov_to_2021Feb("en_climate_daily_ON_6106001_2021_P1D.csv"), # OTTAWA INTL A weather 2021 
          weather_from_2020Nov_to_2021Feb("en_climate_daily_ON_6158731_2020_P1D.csv"), # TORONTO INTL A weather 2020 
          weather_from_2020Nov_to_2021Feb("en_climate_daily_ON_6158731_2021_P1D.csv")] # TORONTO INTL A weather 2020 

source_weather = pd.concat(frames)

In [3]:
source_weather.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
305,-75.67,45.32,OTTAWA INTL A,6106001,2020-11-01,2020,11,1,NaN,7.8,...,0.0,NaN,2.2,NaN,NaN,NaN,17.0,NaN,45.0,NaN
306,-75.67,45.32,OTTAWA INTL A,6106001,2020-11-02,2020,11,2,NaN,2.0,...,3.7,NaN,2.4,NaN,0.0,T,32.0,NaN,80.0,NaN
307,-75.67,45.32,OTTAWA INTL A,6106001,2020-11-03,2020,11,3,NaN,0.3,...,0.0,T,0.0,T,3.0,NaN,31.0,NaN,65.0,NaN
308,-75.67,45.32,OTTAWA INTL A,6106001,2020-11-04,2020,11,4,NaN,13.8,...,0.0,NaN,0.0,NaN,3.0,NaN,NaN,M,NaN,M
309,-75.67,45.32,OTTAWA INTL A,6106001,2020-11-05,2020,11,5,NaN,19.3,...,0.0,NaN,0.0,NaN,NaN,NaN,22.0,NaN,48.0,NaN


In [4]:
#change station name to province name
source_weather["Station Name"] = source_weather["Station Name"].map(lambda x: "Ottawa" if (x=="OTTAWA INTL A") else "Toronto")



In [5]:
#create dimensions
pre_Weather_dimension = pd.DataFrame(columns=['Climate_id',
                                          'Daily_high_temperature','Daily_low_temperature', 'Daily_mean_tempera',
                                          'Total_precip','Total_rain','Total_snow',
                                          'Direction_max_gust','Speed_max_gust',
                                          'Cooling_degree_days','Heating_degree_days',
                                          'Longitude (x)','Latitude (y)','date'])#attributes saved for later join and need removal




In [6]:
#append dimensions
for idx, row in source_weather.iterrows():
    #Weather
    weather_row = []
    
    weather_row.append(row['Climate ID'])
    
    weather_row.append(row['Max Temp (°C)'])
    weather_row.append(row['Min Temp (°C)'])
    weather_row.append(row['Mean Temp (°C)'])
    
    weather_row.append(row['Total Precip (mm)'])
    weather_row.append(row['Total Rain (mm)'])
    weather_row.append(row['Total Snow (cm)'])
    
    weather_row.append(row['Dir of Max Gust (10s deg)'])
    weather_row.append(row['Spd of Max Gust (km/h)'])
    
    weather_row.append(row['Cool Deg Days (°C)'])
    weather_row.append(row['Heat Deg Days (°C)'])
    
    weather_row.append(row['Longitude (x)'])
    weather_row.append(row['Latitude (y)'])
    weather_row.append(row['Date/Time'])
    
    pre_Weather_dimension.loc[len(pre_Weather_dimension)] = weather_row
    

In [7]:
#data quality
def data_interpolation(dimension,attribute):
    dimension[attribute] = dimension[attribute].interpolate(method='linear')
    dimension[attribute] = round(dimension[attribute])

attributes = ['Daily_high_temperature','Daily_low_temperature', 'Daily_mean_tempera',
              'Total_precip','Total_rain','Total_snow',
              'Direction_max_gust','Speed_max_gust',
              'Cooling_degree_days','Heating_degree_days']
for a in attributes:
    data_interpolation(pre_Weather_dimension,a)

In [8]:
pre_Weather_dimension['date'] = pd.to_datetime(pre_Weather_dimension['date']).dt.strftime('%Y-%m-%d')
pre_Weather_dimension.insert(0,'Weather_key',np.arange(len(pre_Weather_dimension)))
pre_Weather_dimension.head()

,Weather_key,Climate_id,Daily_high_temperature,Daily_low_temperature,Daily_mean_tempera,Total_precip,Total_rain,Total_snow,Direction_max_gust,Speed_max_gust,Cooling_degree_days,Heating_degree_days,Longitude (x),Latitude (y),date
0,0,6106001,8.0,-1.0,4.0,2.0,2.0,0.0,17.0,45.0,0.0,14.0,-75.67,45.32,2020-11-01
1,1,6106001,2.0,-2.0,-0.0,2.0,0.0,4.0,32.0,80.0,0.0,18.0,-75.67,45.32,2020-11-02
2,2,6106001,0.0,-4.0,-2.0,0.0,0.0,0.0,31.0,65.0,0.0,20.0,-75.67,45.32,2020-11-03
3,3,6106001,14.0,-3.0,5.0,0.0,0.0,0.0,26.0,56.0,0.0,13.0,-75.67,45.32,2020-11-04
4,4,6106001,19.0,-1.0,9.0,0.0,0.0,0.0,22.0,48.0,0.0,9.0,-75.67,45.32,2020-11-05


In [9]:
Weather_dimension = pre_Weather_dimension[['Climate_id',
                                             'Daily_high_temperature','Daily_low_temperature', 'Daily_mean_tempera',
                                             'Total_precip','Total_rain','Total_snow',
                                             'Direction_max_gust','Speed_max_gust',
                                             'Cooling_degree_days','Heating_degree_days',]]
Weather_dimension.head()

,Climate_id,Daily_high_temperature,Daily_low_temperature,Daily_mean_tempera,Total_precip,Total_rain,Total_snow,Direction_max_gust,Speed_max_gust,Cooling_degree_days,Heating_degree_days
0,6106001,8.0,-1.0,4.0,2.0,2.0,0.0,17.0,45.0,0.0,14.0
1,6106001,2.0,-2.0,-0.0,2.0,0.0,4.0,32.0,80.0,0.0,18.0
2,6106001,0.0,-4.0,-2.0,0.0,0.0,0.0,31.0,65.0,0.0,20.0
3,6106001,14.0,-3.0,5.0,0.0,0.0,0.0,26.0,56.0,0.0,13.0
4,6106001,19.0,-1.0,9.0,0.0,0.0,0.0,22.0,48.0,0.0,9.0


In [10]:
csv_result_file_path = 'csv_result_files/'
pre_Weather_dimension.to_csv(csv_result_file_path + 'pre_Weather_dimension.csv',index=False)
Weather_dimension.to_csv(csv_result_file_path + 'Weather_dimension.csv',index=False)